# Парсим данные hh.ru

### Создаем файл json, куда записываем интересующую информацию

In [3]:
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time
import datetime
import json

def get_links(text):
    ua = fake_useragent.UserAgent()
    page = 0
    url = f"https://krasnodar.hh.ru/search/vacancy?text={text}&from=suggest_post&items_on_page=20&search_field=name&page={page}" 
    print("Первая страница поиска:", url)
    data = requests.get(url, headers={"user-agent":ua.random})
    if data.status_code != 200:
        return
    soup = BeautifulSoup(data.content, "lxml")
    
    # Считает сколько страниц результатов
    try:
        page_count = int(soup.find("div", attrs={"class":"pager"}).find_all("span",recursive=False)[-1].find("a").find("span").text)   
        print("Число страниц поиска:", page_count) # по 20 результатов на каждой, иначе парсить придется по-другому    
    except:
        return
    
    # С каждой страницы результатов собирает и выводит ссылки 
    url_count = 0
    for page in range(page_count):
        try:
            data = requests.get(url, headers={"user-agent": ua.random})
            if data.status_code !=200:
                continue
            soup = BeautifulSoup(data.content, "lxml")
            for a in soup.find_all("a", attrs={"class":"serp-item__title"}):
                url_count+=1
                yield f'{a.attrs["href"]}'
        except Exception as e:
            print(f"{e}")
        time.sleep(1)
    print("Количество полученных адресов вакансий:", url_count)     
        
        
def get_vacancy(link):
    ua = fake_useragent.UserAgent()
    data = requests.get(
        url=link,
        headers={"user-agent":ua.random}
    )
    if data.status_code !=200:
        pass
    soup = BeautifulSoup(data.content,"lxml")
    try:
        name = soup.find(attrs={"class":"serp-item__title"}).text
    except:
        name = ""
    try:
        salary = soup.find("span",attrs={"data-qa":"vacancy-serp__vacancy-compensation"}).text.replace("\u202f"," ")
    except:
        salary = ""
    try:
        tags = [tag.text for tag in soup.find(attrs={"class":"bloko-tag-list"}).find_all("span",attrs={"class":"bloko-tag__section_text"})]
    except:                                                                              
        tags = []
    vacancy = {
        "name":name,
        "salary":salary,
        "tags":tags, 
        "url":link
    }
    return vacancy

if __name__ == "__main__": 
    today = datetime.datetime.today()
    # В имени не должно быть "/" и других запрещенных символов
    text = 'Data Scientist' # Это то, что мы вбиваем в поисковую строку hh.ru
    count = 0
    data = []
    for a in get_links(text):
        data.append(get_vacancy(a))
        time.sleep(1)
        with open(f"{text}_{today:%Y.%m.%d_%H.%M.%S}.json","w", encoding="utf-8") as f:
            
            # Добывляем отступ и отключаем проверку,
            # Чтобы русские буквы не заменялись английскими
            json.dump(data, f, indent=4, ensure_ascii=False)
            count += 1
            print(f"обработано запросов", count)
            
            if count == 10000: # вводим количество запросов. Для теста достаточно 3.
                break    
    

    


Первая страница поиска: https://krasnodar.hh.ru/search/vacancy?text=Data Scientist&from=suggest_post&items_on_page=20&search_field=name&page=0
Число страниц поиска: 20
обработано запросов 1
обработано запросов 2
обработано запросов 3
обработано запросов 4
обработано запросов 5
обработано запросов 6
обработано запросов 7
обработано запросов 8
обработано запросов 9
обработано запросов 10
обработано запросов 11
обработано запросов 12
обработано запросов 13
обработано запросов 14
обработано запросов 15
обработано запросов 16
обработано запросов 17
обработано запросов 18
обработано запросов 19
обработано запросов 20
обработано запросов 21
обработано запросов 22
обработано запросов 23
обработано запросов 24
обработано запросов 25
обработано запросов 26
обработано запросов 27
обработано запросов 28
обработано запросов 29
обработано запросов 30
обработано запросов 31
обработано запросов 32
обработано запросов 33
обработано запросов 34
обработано запросов 35
обработано запросов 36
обработано за

обработано запросов 340
обработано запросов 341
обработано запросов 342
обработано запросов 343
обработано запросов 344
обработано запросов 345
обработано запросов 346
обработано запросов 347
обработано запросов 348
обработано запросов 349
обработано запросов 350
обработано запросов 351
обработано запросов 352
обработано запросов 353
обработано запросов 354
обработано запросов 355
обработано запросов 356
обработано запросов 357
обработано запросов 358
обработано запросов 359
обработано запросов 360
обработано запросов 361
обработано запросов 362
обработано запросов 363
обработано запросов 364
обработано запросов 365
обработано запросов 366
обработано запросов 367
обработано запросов 368
обработано запросов 369
обработано запросов 370
обработано запросов 371
обработано запросов 372
обработано запросов 373
обработано запросов 374
обработано запросов 375
обработано запросов 376
обработано запросов 377
обработано запросов 378
обработано запросов 379
обработано запросов 380
обработано запро

# Работа с полученным json

In [4]:
# Пример нормализации данных json
# https://habr.com/ru/companies/otus/articles/731844

import pandas as pd
import json

df = pd.read_json('Data Scientist_2023.08.12_11.52.16.json')
df['name'] = df['name'].str.lower()

# Чтобы развернуть столбец tags, который содержит список строк, применим метод explode
exploded_df = df.explode('tags',ignore_index = True)
exploded_df['tags'] = exploded_df['tags'].str.lower()

# Объединяю синонимы в навыках. Заменяю значения на самый популярный синоним
# ---------------------------------------------------------------------------
for tag in ["machine learning",
            "машинное обучение",
            "методы машинного обучения",
            ]:
    exploded_df.loc[exploded_df["tags"] == tag] = "ml"
    
for tag in ["data analysis",
            ]:
    exploded_df.loc[exploded_df["tags"] == tag] = "анализ данных"
    
for tag in ["статистический анализ",
            "mathematics",
            ]:
    exploded_df.loc[exploded_df["tags"] == tag] = "математическая статистика"    
# ---------------------------------------------------------------------------

# Считаю частоту навыков
tags_freq = exploded_df['tags'].value_counts()

# Считаю частоту использования имени вакансии
name_freq = df['name'].value_counts()

# Считаю среднее число навыков на вакансию
sum_tags = exploded_df.shape[0] # кол. строк = числу навыков
sum_names = df.shape[0] # кол. строк = числу вакансий

print("Среднее число навыков на вакансию: ", round((sum_tags / sum_names), 1))
print ('************')
print (tags_freq) # выводим частоту использования тегов
print ('************')
print (name_freq)# выводим частоту использования названий вакансий

Среднее число навыков на вакансию:  6.3
************
ml                                      487
python                                  287
sql                                     147
pytorch                                 140
data science                            100
математическая статистика                87
tensorflow                               73
pandas                                   67
linux                                    60
английский — b2 — средне-продвинутый     53
hadoop                                   47
numpy                                    47
scikit-learn                             47
nlp                                      47
анализ данных                            47
big data                                 40
airflow                                  40
английский язык                          40
английский — c1 — продвинутый            40
computer vision                          33
presto                                   27
hive                   